In [1]:
from google.colab import drive
drive.mount("/F")

Mounted at /F


In [2]:
import pandas as pd

In [3]:
base = pd.read_csv('/F/My Drive/Maratona/Regressão Linear/house_prices.csv')

In [4]:
base.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [5]:
base.shape

(21613, 21)

In [6]:
X = base.iloc[:, 5:6].values
y = base.iloc[:, 2:3].values

In [7]:
X

array([[1180],
       [2570],
       [ 770],
       ...,
       [1020],
       [1600],
       [1020]])

In [8]:
y

array([[221900.],
       [538000.],
       [180000.],
       ...,
       [402101.],
       [400000.],
       [325000.]])

In [9]:
from sklearn.preprocessing import StandardScaler
scaler_x = StandardScaler()
X = scaler_x.fit_transform(X)
scaler_y = StandardScaler()
y = scaler_y.fit_transform(y)

In [10]:
X

array([[-0.97983502],
       [ 0.53363434],
       [-1.42625404],
       ...,
       [-1.15404732],
       [-0.52252773],
       [-1.15404732]])

In [11]:
y

array([[-0.86671733],
       [-0.00568792],
       [-0.98084935],
       ...,
       [-0.37586519],
       [-0.38158814],
       [-0.58588173]])

In [12]:
import tensorflow as tf

In [13]:
colunas = [tf.feature_column.numeric_column('previsor', shape = [1])]

In [14]:
colunas

[NumericColumn(key='previsor', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [15]:
config = tf.estimator.RunConfig(tf_random_seed = 42)

In [16]:
regressor = tf.estimator.LinearRegressor(feature_columns = colunas, config = config)

INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp7obyzwyu', '_tf_random_seed': 42, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [17]:
from sklearn.model_selection import train_test_split
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [18]:
X_treinamento.shape

(15129, 1)

In [19]:
y_treinamento.shape

(15129, 1)

In [20]:
X_teste.shape

(6484, 1)

In [21]:
y_teste.shape

(6484, 1)

In [22]:
def make_input_fn(X, y = None, num_epochs = None, shuffle = True, batch_size = 32):
    def input_function():
        # Precisamos criar um dataset com as variáveis previsoras e de saída. 
        # As variáveis previsoras precisam ser passadas como um dicionário com os mesmos nomes definidos na chamada de `tf.feature_column.numeric_column`
        ds = tf.data.Dataset.from_tensor_slices(({'previsor': X}, y))
        if shuffle:
            ds = ds.shuffle(1000)
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    return input_function

In [23]:
funcao_treinamento = make_input_fn(X_treinamento, y_treinamento)

In [24]:
funcao_teste = make_input_fn(X_teste, y_teste, num_epochs = 1000, shuffle = False)

In [25]:
regressor.train(input_fn = funcao_treinamento, steps = 10000)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp7obyzwyu/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 0.89202815, step = 0
INFO:tensorflow:global_step/sec: 778.004
INFO:tensorflow:loss = 0.3037456, step = 100 (0.130 sec)
INFO:tensorflow:global_step/sec: 916.171
INFO:tensorflow:loss = 0.46357384, step = 200 (0.113 sec)
INFO:tensorflow:global_step/sec: 999.115
INFO:tensorflow:loss = 0.241681, step = 300 (0.099 sec)
INFO:tensorflow:global_step/sec: 1050.49
INFO:tensorflow:loss = 0.37792256, step = 400 (0.094 sec)
INFO:tensorflow:gl

In [26]:
metricas_treinamento = regressor.evaluate(input_fn = funcao_treinamento, steps = 10000)

INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-01-26T19:40:15Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp7obyzwyu/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1000/10000]
INFO:tensorflow:Evaluation [2000/10000]
INFO:tensorflow:Evaluation [3000/10000]
INFO:tensorflow:Evaluation [4000/10000]
INFO:tensorflow:Evaluation [5000/10000]
INFO:tensorflow:Evaluation [6000/10000]
INFO:tensorflow:Evaluation [7000/10000]
INFO:tensorflow:Evaluation [8000/10000]
INFO:tensorflow:Evaluation [9000/10000]
INFO:tensorflow:Evaluation [10000/10000]
INFO:tensorflow:Inference Time : 7.01941s
INFO:tensorflow:Finished evaluation at 2021-01-26-19:40:23
INFO:tensorflow:Saving dict for global step 10000: average_loss = 0.48810703, global_step = 10000, label/mean = -0.0065834196, loss = 0.48807672, prediction/mean = -0.012411411
INFO:tensorflow:Saving 'ch

In [27]:
metricas_teste = regressor.evaluate(input_fn = funcao_teste, steps = 10000)

INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-01-26T19:41:13Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp7obyzwyu/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1000/10000]
INFO:tensorflow:Evaluation [2000/10000]
INFO:tensorflow:Evaluation [3000/10000]
INFO:tensorflow:Evaluation [4000/10000]
INFO:tensorflow:Evaluation [5000/10000]
INFO:tensorflow:Evaluation [6000/10000]
INFO:tensorflow:Evaluation [7000/10000]
INFO:tensorflow:Evaluation [8000/10000]
INFO:tensorflow:Evaluation [9000/10000]
INFO:tensorflow:Evaluation [10000/10000]
INFO:tensorflow:Inference Time : 6.56078s
INFO:tensorflow:Finished evaluation at 2021-01-26-19:41:19
INFO:tensorflow:Saving dict for global step 10000: average_loss = 0.55330914, global_step = 10000, label/mean = 0.015570625, loss = 0.55679685, prediction/mean = 0.0032317282
INFO:tensorflow:Saving 'chec

In [28]:
metricas_treinamento

{'average_loss': 0.48810703,
 'global_step': 10000,
 'label/mean': -0.0065834196,
 'loss': 0.48807672,
 'prediction/mean': -0.012411411}

In [29]:
metricas_teste

{'average_loss': 0.55330914,
 'global_step': 10000,
 'label/mean': 0.015570625,
 'loss': 0.55679685,
 'prediction/mean': 0.0032317282}

In [30]:
import numpy as np
novas_casas = np.array([[800], [900], [1000]])
novas_casas

array([[ 800],
       [ 900],
       [1000]])

In [31]:
novas_casas = scaler_x.transform(novas_casas)
novas_casas

array([[-1.39358923],
       [-1.28470655],
       [-1.17582386]])

In [32]:
funcao_previsao = make_input_fn(novas_casas, num_epochs = 1, shuffle = False)

In [33]:
previsoes = regressor.predict(input_fn = funcao_previsao)

In [34]:
previsoes

<generator object Estimator.predict at 0x7f6f7efb14c0>

In [35]:
list(previsoes)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp7obyzwyu/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


[{'predictions': array([-0.96557844], dtype=float32)},
 {'predictions': array([-0.8907409], dtype=float32)},
 {'predictions': array([-0.8159032], dtype=float32)}]

In [36]:
for p in regressor.predict(input_fn = funcao_previsao):
    print(scaler_y.inverse_transform(p['predictions']))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp7obyzwyu/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[185606.23]
[213080.52]
[240554.83]


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '
